Agentlar çevresinden bilgi alıp (girdi), bu bilgiye göre karar verip (işlem), çevreye bir tepki (çıktı) verebilen bir varlıktır.
Bir AI agent ise, yapay zekâ algoritmalarıyla çalışan bir ajandır. Yani:

- Durumu analiz eder (örneğin: doğal dil anlama, görüntü tanıma).

- Kendi başına karar alır.

- Belirli görevleri yerine getirir (örneğin: web'den bilgi toplar, e-posta gönderir, veritabanına veri yazar).

In [9]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://blog.langchain.dev/langgraph-platform-ga/")

docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
# Sayısal temsillere çevir
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter 

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

In [ ]:
# Sayısal temsillere çevrildi
from langchain_community.vectorstores import FAISS

vector = FAISS.from_documents(documents, embeddings)

In [6]:
retriever = vector.as_retriever()

In [7]:
retriever.get_relevant_documents(
    "What is LangGraph?"
)[0]

C:\Users\Vedat\AppData\Local\Temp\ipykernel_22788\916090378.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents(


Document(id='2a1011f9-cb0d-4edf-8d6e-cd20e2d735bc', metadata={'source': 'https://blog.langchain.dev/langgraph-platform-ga/', 'title': 'LangGraph Platform is now Generally Available: Deploy & manage long-running, stateful Agents', 'description': 'LangGraph Platform, our infrastructure for deploying and managing agents at scale, is now generally available. Learn how to deploy', 'language': 'en'}, page_content='management console makes it easier to enforce consistency, monitor behavior, and ship updates safely — all without needing to re-deploy or touch code every time. You can:Discover available agents in the agent registryCreate different versions of your agent (“assistants”) in LangGraph platform, allowing you to reuse common agent architectures\xa0Leverage other agents as “Remote Graphs”, allowing you to create multi agent architectures that run in a distributed mannerFor companies like Qualtrics, centralizing agent management with LangGraph Platform has been critical to driving effic

In [8]:
# Creating Retriever Tool
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever, 
    "langgraph_search",
    "Search for information about LangGraph. For any question about LangGraph, you must use this tool!"
)

In [10]:
# Creating Search Tool(Tavily) = Özellikle agentlar için oluşturulmuş bir tool
from langchain_community.tools.tavily_search import TavilySearchResults 

search = TavilySearchResults()


In [11]:
# Deneme amaçlı
search.invoke("What is the weather in İstanbul?")

[{'title': 'İstanbul weather in May 2025 ⋆ İstanbul temperature in May ...',
  'url': 'https://www.meteoprog.com/weather/Istanbul/month/may/',
  'content': 'İstanbul (Turkey) weather in May 2025 ☀️ Accurate weather forecast for İstanbul in ... 21 May. +21°+15°. 22 May. +21°+16°. 23 May. +22°+16°. 24 May. +21°+16°. 25',
  'score': 0.9163571},
 {'title': 'Istanbul weather in May 2025 - Weather25.com',
  'url': 'https://www.weather25.com/europe/turkey/istanbul?page=month&month=May',
  'content': 'The temperatures in Istanbul in May are comfortable with low of 15°C and and high up to 22°C.\n\nYou can expect about 3 to 8 days of rain in Istanbul during the month of May. It’s a good idea to bring along your umbrella so that you don’t get caught in poor weather.\n\nOur weather forecast can give you a great sense of what weather to expect in Istanbul in May 2025. [...] Istanbul weather in May 2025\n\nSun | Mon | Tue | Wed | Thu | Fri | Sat\n |  |  |  | 112°/10° | 213°/10° | 317°/12°\n416°/13° 

In [23]:
tools = [retriever_tool, search]

In [24]:
# Creating Agent
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

In [25]:
from langchain import hub 

prompt = hub.pull("hwchase17/openai-functions-agent")

c:\Users\Vedat\anaconda3\envs\Langchain\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [26]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [31]:
# Initializing the Agent
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt)

In [32]:
# Agentin aksiyonları yapması için import edilecek modül
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)


In [33]:
# Executing the Agent
agent_executor.invoke(
    {
        "input":"Hi!"
    }
)



> Entering new AgentExecutor chain...
Hello! How can I help you today?


> Finished chain.


{'input': 'Hi!', 'output': 'Hello! How can I help you today?\n'}

In [34]:
# Executing the Agent
agent_executor.invoke(
    {
        "input": "What is the weather in London?"
    }
)




> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in London'}`


[{'title': 'England weather forecast for Wednesday, 21 May 2025', 'url': 'https://www.weather25.com/europe/united-kingdom?page=date&date=21-5', 'content': "| 00:00 | 03:00 | 06:00 | 09:00 | 12:00 | 15:00 | 18:00 | 21:00\nTemperature | 13°C | 12°C | 13°C | 15°C | 20°C | 21°C | 18°C | 13°C\nWeather | Overcast | Cloudy | Overcast | Cloudy | Partly cloudy | Partly cloudy | Sunny | Clear\nChance of rain | 0% | 0% | 0% | 0% | 0% | 0% | 0% | 0%\nPrecipitation | 0mm | 0mm | 0mm | 0mm | 0mm | 0mm | 0mm | 0mm\nHumidity | 71% | 71% | 65% | 51% | 36% | 32% | 46% | 49%\nWind | 2Km/h | 1Km/h | 5Km/h | 5Km/h | 10Km/h | 13Km/h | 13Km/h | 9Km/h [...] Wind Gust | 3Km/h | 2Km/h | 7Km/h | 6Km/h | 12Km/h | 15Km/h | 17Km/h | 14Km/h\nWind Degree | 42° | 45° | 50° | 9° | 356° | 12° | 36° | 36°\nWind Direction | NE | NE | NE | N | N | NNE | NE | NE\nCloud Cover | 93% | 68% | 99% | 70% | 32% |

{'input': 'What is the weather in London?',
 'output': 'The weather in London in May 2025 is expected to have temperatures ranging from 8°C to 16°C.  There will be some rain, with estimates varying from 3 to 8 days of rain, and a total of around 68mm of precipitation.  More detailed forecasts show a mix of overcast, cloudy, partly cloudy, sunny, and clear days, with some days having moderate or heavy rain showers.  Specific daily forecasts vary depending on the source.\n'}

In [35]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(
        content="Can I use LangGraph for agent runtimes?"
    ),
    AIMessage(content="Yes!")
]

In [37]:
agent_executor.invoke(
    {
        "chat_history": chat_history, 
        "input": "Tell me how"
    }
)



> Entering new AgentExecutor chain...

Invoking: `langgraph_search` with `{'query': 'agent runtimes'}`


management console makes it easier to enforce consistency, monitor behavior, and ship updates safely — all without needing to re-deploy or touch code every time. You can:Discover available agents in the agent registryCreate different versions of your agent (“assistants”) in LangGraph platform, allowing you to reuse common agent architectures Leverage other agents as “Remote Graphs”, allowing you to create multi agent architectures that run in a distributed mannerFor companies like Qualtrics, centralizing agent management with LangGraph Platform has been critical to driving efficiency: "The future of agentic AI is multi-vendor and AI agents must be built in an ecosystem. By using LangGraph Platform to build and manage our AI agents - Experience Agents - Qualtrics is able to design, deploy, and manage complex generative AI agent workflows with efficiency, speed, and scale." – Phil M

{'chat_history': [HumanMessage(content='Can I use LangGraph for agent runtimes?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Yes!', additional_kwargs={}, response_metadata={})],
 'input': 'Tell me how',
 'output': "LangGraph Platform is a purpose-built infrastructure and management layer for deploying and scaling long-running, stateful agents.  It offers 1-click deployment, horizontal scaling, a persistence layer for memory and history, and a Studio for debugging.  It's available in cloud, hybrid, and self-hosted options.  The platform centralizes agent management, allowing for consistency, monitoring, and safe updates without re-deployment.  It integrates with GitHub and offers various APIs for custom user experiences.  LangGraph Platform can be used independently or with other LangChain products.\n"}

{'chat_history': [HumanMessage(content='Can I use LangGraph for agent runtimes?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Yes!', additional_kwargs={}, response_metadata={})],
 'input': 'Tell me how',
 'output': "LangGraph Platform is a purpose-built infrastructure and management layer for deploying and scaling long-running, stateful agents.  It offers 1-click deployment, horizontal scaling, a persistence layer for memory and history, and a Studio for debugging.  It's available in cloud, hybrid, and self-hosted options.  The platform centralizes agent management, allowing for consistency, monitoring, and safe updates without re-deployment.  It integrates with GitHub and offers various APIs for custom user experiences.  LangGraph Platform can be used independently or with other LangChain products.\n"}